In [11]:
# installing and importing mip
!pip install --user mip
from mip import *
import math

# defining parameters
E = 950000  # yearly electricity usage (W) from user
E0 = E/12 # convert to average monthly electricy usage (W)
E = [E0] # yearly electricy usage (W) with trend
Cg = 0.0134  # cost of electricity from the grid ($/W)
m = 10  # yearly maintenance cost ($/panel)
B = 25000  # budget from user
Cp = 882  # cost of each solar panel ($/panel) (12 modules of 60cell)
Ap = 18.9  # area of solar panel (ft^2) (40 * 68 inches)
Ar = 15000000  # area of the roof (ft^2) from user
Armax = (Ar/2) * 0.8
P = 315*12  # capacity of each solar panel (W)
F = 2500  # fixed costs of installing solar panels
d = []  # deterioration factor at year i (%)
U = 0.98  # how much energy can be used (%) (factors: irradiation, shadow, direction)
I = 25 # lifespan of solar panels

# filling in depreciation values (currently set to 0.05)
for i in range(I):
    d.append(0.05 * i)

# function to fill in Et - linearly decreases by 1.4%
for i in range(1,I):
    E.append(E[i-1] - (E[i-1]*0.014))

# filling in cost of electricity and converting to present value Cgt - TODO

# actually calculate U properly
# define variables
soiling = 0.02 # loses due to dirt/dust on panels (higher for high pollution, low rainfall areas)
shading = 0.03 # might change to be an input from users
snow = 0 # assume that homeowner will clear snow from panels
mismatch = 0.02 # electrical losses due to manufacturing
wiring = 0.02 # loss between DC and AC conversion
connections = 0.005 # loss between electrical connectors
nameplaterating = 0.01
availability = 0.03

U = (1-soiling) * (1-shading) * (1-snow) * (1-mismatch) * (1-wiring) * (1-connections) * (1-nameplaterating) * (1-availability)
    
# convert m into present value - TODO
# FV = PV * ((1 + (i/n))^(n*t))

# using E @ t=0
Pn = math.ceil(E0/(P*U)) # number of solar panels needed to fulfill at least 100% of electricity from the grid

# initializing model
model = Model()

# initializing decision variable
y = model.add_var(name='y', var_type=INTEGER)  # number of solar panels

# initializing the objective function
# currently only looking at a time horizon of one year
#model.objective = minimize(xsum(((E - ((y * P * U) * (1 - d[i]))) * Cg) + (25 * m * y) for i in range(I)))
model.objective = minimize(xsum((E[i] - ((y*P*U) * (1-d[i]))) * Cg + (m * y) for i in range(I)))

# adding constraints
model += (y * Cp) + F <= B  # budget constraint
model += y * Ap <= Armax  # area of roof constraint **NEED TO ADD FACTOR TO CALCULATE USABLE ROOF SIZE**
model += Pn - y >= 0 # fufill demand contraint
model += y >= 0  # non-negativity constraint

# solving the MIP
status = model.optimize()

# printing solution
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print("Z = {}".format(model.objective_value))
    print("y = {}".format(y.x))  # printing decision variable value
    numWatts = (y.x) * P
    numPanels = y.x
    print("optimal number of watts to install is: " + str(numWatts) + " W")
    if numPanels > 0:
        totalCost = (numPanels * Cp) + F
    else:
        totalCost = 0
    print("Total Captial Cost: $" + str(totalCost))
if status == OptimizationStatus.NO_SOLUTION_FOUND:
    print("no feasible solution :(")


Z = 17712.508172642705
y = 25.0
optimal number of watts to install is: 94500.0 W
Total Captial Cost: $24550.0
